# Imports

In [1]:
import cv2
import numpy as np
import pandas as pd
import time
import pickle
import PIL
import os
import winsound
import glob

from matplotlib import pyplot as plt

from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime

from collections import deque

# Recording data from Kinect v2

In [ ]:
def play_beep(duration):
    winsound.Beep(2500, duration)


In [ ]:
kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth)
kinect.close()

In [ ]:
def reshape_frames(color_frames, depth_frames):
    reshaped_color_frames = deque(maxlen=60)
    reshaped_depth_frames = deque(maxlen=60)

    for color_frame in color_frames:
        reshaped_color_frame = color_frame.reshape((kinect.color_frame_desc.Height, kinect.color_frame_desc.Width, -1)).astype(np.uint8)
        reshaped_color_frame = cv2.resize(reshaped_color_frame, (681,383))
        reshaped_color_frame = reshaped_color_frame[:,158:541,:]
        reshaped_color_frames.append(reshaped_color_frame)

    for depth_frame in depth_frames:
        reshaped_depth_frame = depth_frame.reshape((kinect.depth_frame_desc.Height, kinect.depth_frame_desc.Width)).astype(np.uint16)
        reshaped_depth_frame = reshaped_depth_frame[16:399,64:447]
        reshaped_depth_frames.append(reshaped_depth_frame)

    return reshaped_color_frames, reshaped_depth_frames

def save_to_npz(name, action, color_frames, depth_frames):
    color_file_path = f'./numpy_data_arrays/{action}/{name}_color.npz'
    depth_file_path = f'./numpy_data_arrays/{action}/{name}_depth.npz'
    np.savez_compressed(color_file_path, np.array(color_frames))
    np.savez_compressed(depth_file_path, np.array(depth_frames))

def record_to_video_files(name, action, color_frames, depth_frames):
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    color_file_path = f'./videos/{action}/{name}_color.mov'
    depth_file_path = f'./videos/{action}/{name}_depth.mov'

    color_video_writer = cv2.VideoWriter(color_file_path, fourcc, len(color_frames) // 2, (383, 383))
    depth_video_writer = cv2.VideoWriter(depth_file_path, fourcc, len(depth_frames) // 2, (383, 383), isColor=False)

    for color_frame in color_frames:
        frame_to_add = cv2.cvtColor(color_frame, cv2.COLOR_RGBA2RGB)
        color_video_writer.write(frame_to_add)
    color_video_writer.release()

    for depth_frame in depth_frames:
        frame_to_add = depth_frame.astype(np.uint8)

        depth_video_writer.write(frame_to_add)

    depth_video_writer.release()

    return None

def record_new_rgbd_data(name, action):
    kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth)
    color_frames = deque(maxlen=60)  
    depth_frames = deque(maxlen=60)
    timeStart = time.time()
    # WARM START / Countdown - 8 seconds of color and depth
    while time.time() < timeStart + 5:
        if kinect.has_new_color_frame():
            color_frame = kinect.get_last_color_frame()
            depth_frame = kinect.get_last_depth_frame()
    play_beep(200)
    # Record 2 seconds of color and depth
    while time.time() < timeStart + 7:
        if kinect.has_new_color_frame():
            color_frame = kinect.get_last_color_frame()
            depth_frame = kinect.get_last_depth_frame()
            color_frames.append(color_frame)
            depth_frames.append(depth_frame)
    play_beep(600)
    
    kinect.close()
    print("RECORDING DONE")
    reshaped_color_frames, reshaped_depth_frames = reshape_frames(color_frames, depth_frames)
    print("FRAMES RESHAPED")
    save_to_npz(name, action, reshaped_color_frames, reshaped_depth_frames)
    print("SAVED TO NPZ")
    record_to_video_files(name, action, reshaped_color_frames, reshaped_depth_frames)
    print("VIDEO RECORDED")

    return None

In [ ]:
# for i in range(1, 26):
#     print("JAB NUMBER", i)
#     record_new_rgbd_data(f'jab_{i}', 'jab')

In [ ]:
# for i in range(1, 26):
#     print("CROSS NUMBER", i)
#     record_new_rgbd_data(f'cross_{i}', 'cross')

In [ ]:
# for i in range(1, 26):
#     print("LEFT HOOK NUMBER", i)
#     record_new_rgbd_data(f'left_hook_{i}', 'left_hook')

In [ ]:
# for i in range(1, 26):
#     print("RIGHT HOOK NUMBER", i)
#     record_new_rgbd_data(f'right_hook_{i}', 'right_hook')

In [ ]:
# for i in range(1, 26):
#     print("RANDOM NUMBER", i)
#     record_new_rgbd_data(f'random_{i}', 'random')

# MHI

In [3]:
THRESHOLD_VALUE = 5
MHI_DURATION = 2

def generate_mhi(frames):
    number_of_frames = frames.shape[0]
    height = frames.shape[1]
    width = frames.shape[2]
    SAMHI_10 = np.zeros((height, width), dtype=np.float32)

    for i in range(1, number_of_frames):
        frame = frames[i]
        frame[frame > 2200] = 0

        image_binary = frame.astype(np.uint8)

        num = 5
        image_binary_prev = np.zeros((height, width), dtype=np.uint8)
        difference = np.zeros((height, width), dtype=np.uint8)

        if i == 1:
            image_binary_prev = image_binary
        elif (i % num) == 0:
            difference = cv2.absdiff(image_binary_prev, image_binary)
            image_binary_prev = image_binary

        if i == num + 1:
            _, image_binary_diff_5 = cv2.threshold(difference, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
            SAMHI_10 = cv2.motempl.updateMotionHistory(image_binary_diff_5, SAMHI_10, i / number_of_frames, MHI_DURATION)

        if (i > num + 1 and i % num == 0):
            _, image_binary_diff_5 = cv2.threshold(difference, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
            SAMHI_10 = cv2.motempl.updateMotionHistory(image_binary_diff_5, SAMHI_10, i / number_of_frames, MHI_DURATION)


    SAMHI_10 = cv2.convertScaleAbs(SAMHI_10, alpha=255, beta=0)
    return SAMHI_10



In [4]:
jab_depth_files = [file for file in glob.glob('./numpy_data_arrays/jab/*depth.npz')]
cross_depth_files = [file for file in glob.glob('./numpy_data_arrays/cross/*depth.npz')]
left_hook_depth_files = [file for file in glob.glob('./numpy_data_arrays/left_hook/*depth.npz')]
right_hook_depth_files = [file for file in glob.glob('./numpy_data_arrays/right_hook/*depth.npz')]
random_depth_files = [file for file in glob.glob('./numpy_data_arrays/random/*depth.npz')]

for file in jab_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    cv2.imwrite(f'./mhi/jab/{file_name}.png', samhi)

for file in cross_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    cv2.imwrite(f'./mhi/cross/{file_name}.png', samhi)

for file in left_hook_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    cv2.imwrite(f'./mhi/left_hook/{file_name}.png', samhi)

for file in right_hook_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    cv2.imwrite(f'./mhi/right_hook/{file_name}.png', samhi)

for file in random_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    cv2.imwrite(f'./mhi/random/{file_name}.png', samhi)

# SIFT

In [ ]:
def extract_sift_features(mhis):
    keypoints_list = []
    descriptors_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for mhi in mhis:
        keypoints, descriptors = sift.detectAndCompute(mhi, None)
        if descriptors is not None:  # Check if descriptors are None
            keypoints_list.append(keypoints)
            descriptors_list.append(descriptors)

    return keypoints_list, descriptors_list

In [ ]:
jab_mhi_files = [file for file in glob.glob('./mhi/jab/*.png')]
jab_mhis = [cv2.imread(jab_mhi) for jab_mhi in jab_mhi_files]
jab_mhi_features = extract_sift_features(jab_mhis)

cross_mhi_files = [file for file in glob.glob('./mhi/cross/*.png')]
cross_mhis = [cv2.imread(cross_mhi) for cross_mhi in cross_mhi_files]
cross_mhi_features = extract_sift_features(cross_mhis)

left_hook_mhi_files = [file for file in glob.glob('./mhi/left_hook/*.png')]
left_hook_mhis = [cv2.imread(left_hook_mhi) for left_hook_mhi in left_hook_mhi_files]
left_hook_mhi_features = extract_sift_features(left_hook_mhis)

right_hook_mhi_files = [file for file in glob.glob('./mhi/right_hook/*.png')]
right_hook_mhis = [cv2.imread(right_hook_mhi) for right_hook_mhi in right_hook_mhi_files]
right_hook_mhi_features = extract_sift_features(right_hook_mhis)

random_mhi_files = [file for file in glob.glob('./mhi/random/*.png')]
random_mhis = [cv2.imread(random_mhi) for random_mhi in random_mhi_files]
random_mhi_features = extract_sift_features(random_mhis)


In [ ]:
def visualize_sift_features(images, features):
    for i in range(len(images)):
        img = images[i].copy()
        keypoints, descriptors = features[i]
        for kp in keypoints:
            x, y = kp.pt
            cv2.circle(img, (int(x), int(y)), 2, (0, 255, 0), -1)
        cv2.imshow(f"Image {i+1}", img)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

visualize_sift_features(jab_mhis, jab_mhi_features)